# 함수

In [52]:
# 라이브러리
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString
import matplotlib.pyplot as plt
import matplotlib
import warnings
import requests, json
warnings.filterwarnings('ignore')

In [121]:
# DB 
data_frame_DB = clear_input_df()
data_frame_DB

,단지명,전용면적,층,도로명주소
0,0,0.0,0,0


In [ ]:
def add_to_DB(df_row):
    

In [111]:
# 데이터 프레임 초기와
def clear_input_df():
    cnt = 0
    # input_df = pd.DataFrame(columns=range(4))
    # input_df.columns = ['단지명','전용면적','층','도로명주소']
    idx = [cnt]
    heng = [['0',0.0,'0','0']]
    columns = ['단지명','전용면적','층','도로명주소']
    input_df = pd.DataFrame(data=heng, index = idx, columns=columns )
    return input_df

In [129]:
input_df = clear_input_df()
input_df

,단지명,전용면적,층,도로명주소
0,0,0.0,0,0


In [82]:
# 데이터 가져오기
metro = pd.read_csv('https://raw.githubusercontent.com/8900j/hhh/main/%EC%84%9C%EC%9A%B8_%EC%A7%80%ED%95%98%EC%B2%A0_%EC%9C%84%EA%B2%BD%EB%8F%84.csv')
bus = pd.read_csv('https://raw.githubusercontent.com/8900j/hhh/main/%EB%B2%84%EC%8A%A4%EC%A0%95%EB%A5%98%EC%9E%A5_%EC%A4%91%EA%B5%AC.csv')
food = pd.read_csv('https://raw.githubusercontent.com/8900j/hhh/main/%EC%A4%91%EA%B5%AC_%EB%AC%B8%ED%99%94%EA%B4%80%EA%B4%91_%EB%A7%9B%EC%A7%91_%EC%9C%84%EA%B2%BD%EB%8F%84.csv')
munhwa_space = pd.read_csv('https://raw.githubusercontent.com/8900j/hhh/main/%EC%A4%91%EA%B5%AC_%EB%AC%B8%ED%99%94%EA%B4%80%EA%B4%91_%EB%AC%B8%ED%99%94%EA%B3%B5%EA%B0%84_%EC%9C%84%EA%B2%BD%EB%8F%84.csv')
munhwa = pd.read_csv('https://raw.githubusercontent.com/8900j/hhh/main/%EC%A4%91%EA%B5%AC_%EB%AC%B8%ED%99%94%EA%B4%80%EA%B4%91_%EB%AC%B8%ED%99%94%EC%9E%AC_%EC%9C%84%EA%B2%BD%EB%8F%84.csv')
shopping = pd.read_csv('https://raw.githubusercontent.com/8900j/hhh/main/%EC%A4%91%EA%B5%AC_%EB%AC%B8%ED%99%94%EA%B4%80%EA%B4%91_%EC%87%BC%ED%95%91_%EC%9C%84%EA%B2%BD%EB%8F%84.csv')


In [130]:
# 1단계: input_df 데이터프레임 만들기

def make_input_df(data_frame):
    # 빈 DataFrame 생성
    data_frame = clear_input_df()
    # columns = ['단지명','전용면적','층','도로명주소']
    idx = [cnt]
    # heng = [['0',0.0,'0','0']]
    # input_df = pd.DataFrame(data=heng, index = idx, columns=columns )

    # 단지명 , 전용면적, 도로명주소 입력
    apart = input(' 단지명을 입력하세요, ex : 남산센트럴자이 -- ')
    myeonjuk = input(' 전용면적을 입력하세요, ex : 175 -- ')
    floor = input(' 층수를 입력하세요, ex : 21 -- ')
    juso = input(' 도로명 주소를 입력하세요, ex : 중구 퇴계로 235 -- ')
    data_frame.loc[0] = [apart, myeonjuk, floor, juso]
#     try :
#         data_frame.loc[data_frame.index[-1]+1] = [apart, myeonjuk, floor, juso]
#     except :
#         data_frame.loc[0] = [apart, myeonjuk, floor, juso]
    
    # input_df[['단지명','전용면적','층','도로명주소']][cnt] = [apart , myeonjuk, floor, juso]
    return data_frame

In [132]:
tmp = make_input_df(input_df)
print('-------------------------------')
print(tmp)
print(tmp.info())

 단지명을 입력하세요, ex : 남산센트럴자이 --  ㄴ
 전용면적을 입력하세요, ex : 175 --  1
 층수를 입력하세요, ex : 21 --  1
 도로명 주소를 입력하세요, ex : 중구 퇴계로 235 --  중구 퇴계로 235


-------------------------------
  단지명 전용면적  층       도로명주소
0   ㄴ    1  1  중구 퇴계로 235
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1 entries, 0 to 0
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   단지명     1 non-null      object
 1   전용면적    1 non-null      object
 2   층       1 non-null      object
 3   도로명주소   1 non-null      object
dtypes: object(4)
memory usage: 148.0+ bytes
None


In [139]:
# 좌표 변환
def get_location(address):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + address
    headers = {"Authorization": "KakaoAK 2550728cf88fe76f21185b7b79eddac8"}    
    try:      
        api_json = json.loads(str(requests.get(url,headers=headers).text))
        address = api_json['documents'][0]['address']
        crd = {"lat": str(address['y']), "lng": str(address['x'])}
        address_name = address['address_name']
        return crd
    except:
        crd = {"lat": 0 , "lng" : 0 }
        return crd

def set_lat_lng(input_df):
    input_df['위경도'] = input_df['도로명주소'].apply(lambda x : get_location(x))
    input_df['위도'] = input_df['위경도'].apply(lambda x : x['lat'])
    input_df['경도'] = input_df['위경도'].apply(lambda x : x['lng'])
    input_df.drop(columns='위경도', inplace=True)

    input_df[['위도','경도']] = input_df[['위도','경도']].astype(float)

    input_df['geometry'] = input_df.apply(lambda row : Point([row['경도'], row['위도']]), axis= 1)
    input_df = gpd.GeoDataFrame(input_df, geometry='geometry')

    input_df.crs = {'init':'epsg:4326'}
    input_df = input_df.to_crs({'init':'epsg:5179'})
    return input_df


In [140]:
tmp = set_lat_lng(tmp)
tmp

TypeError: 'NoneType' object is not subscriptable

In [124]:
tmp = count_infra(tmp)
tmp

In [105]:
# 2단계 프로토타입

In [108]:
# 좌표 변환
def get_location(address):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + address
    headers = {"Authorization": "KakaoAK 2550728cf88fe76f21185b7b79eddac8"}    
    try:      
        api_json = json.loads(str(requests.get(url,headers=headers).text))
        address = api_json['documents'][0]['address']
        crd = {"lat": str(address['y']), "lng": str(address['x'])}
        address_name = address['address_name']
        return crd
    except:
        crd = {"lat": 0 , "lng" : 0 }
        return crd

def set_lat_lng(input_df):
    input_df['위경도'] = input_df['도로명주소'].apply(lambda x : get_location(x))
    input_df['위도'] = input_df['위경도'].apply(lambda x : x['lat'])
    input_df['경도'] = input_df['위경도'].apply(lambda x : x['lng'])
    input_df.drop(columns='위경도', inplace=True)

    input_df[['위도','경도']] = input_df[['위도','경도']].astype(float)

    input_df['geometry'] = input_df.apply(lambda row : Point([row['경도'], row['위도']]), axis= 1)
    input_df = gpd.GeoDataFrame(input_df, geometry='geometry')

    input_df.crs = {'init':'epsg:4326'}
    input_df = input_df.to_crs({'init':'epsg:5179'})


In [109]:
set_lat_lng(tmp)
tmp

,단지명,전용면적,층,도로명주소,위도,경도,geometry
0,남산,175,21,중구 퇴계로 235,0.0,0.0,POINT (0.00000 0.00000)


In [93]:
# 2단계: 프로토타입

def count_infra(input_df):
    input_df['위경도'] = input_df['도로명주소'].apply(lambda x : get_location(x))
    input_df['위도'] = input_df['위경도'].apply(lambda x : x['lat'])
    input_df['경도'] = input_df['위경도'].apply(lambda x : x['lng'])
    input_df.drop(columns='위경도', inplace=True)

    input_df[['위도','경도']] = input_df[['위도','경도']].astype(float)

    input_df['geometry'] = input_df.apply(lambda row : Point([row['경도'], row['위도']]), axis= 1)
    input_df = gpd.GeoDataFrame(input_df, geometry='geometry')

    input_df.crs = {'init':'epsg:4326'}
    input_df = input_df.to_crs({'init':'epsg:5179'})

    # 지하철
    metro_gdf=gpd.GeoDataFrame(metro,geometry=gpd.points_from_xy(metro.역사위치경도, metro.역사위치위도),crs='epsg:4326')
    def close_metro_dist(point_geometry):
        dist=metro_gdf.to_crs(epsg=5186).geometry.distance(point_geometry)
        return dist.min()
    def close_metro(point_geometry):
        dist=metro_gdf.to_crs(epsg=5186).geometry.distance(point_geometry)
        return metro.iloc[dist.idxmin()]['역명']
    input_df['가장 가까운 지하철역까지 거리(m)']=input_df.to_crs(epsg=5186).apply(lambda x: close_metro_dist(x.geometry),axis=1)
    input_df['가장 가까운 지하철역']=input_df.to_crs(epsg=5186).apply(lambda x: close_metro(x.geometry),axis=1)

    # 버스
    bus_gdf=gpd.GeoDataFrame(bus,geometry=gpd.points_from_xy(bus.정류장_경도, bus.정류장_위도),crs='epsg:4326')
    def close_bus_dist(point_geometry):
        dist=bus_gdf.to_crs(epsg=5186).geometry.distance(point_geometry)
        return dist.min()
    def close_bus(point_geometry):
        dist=bus_gdf.to_crs(epsg=5186).geometry.distance(point_geometry)
        return bus.iloc[dist.idxmin()]['정류장명']
    input_df['가장 가까운 버스정류장까지 거리(m)']=input_df.to_crs(epsg=5186).apply(lambda x: close_bus_dist(x.geometry),axis=1)
    input_df['가장 가까운 버스정류장']=input_df.to_crs(epsg=5186).apply(lambda x: close_bus(x.geometry),axis=1)

    # 버퍼 생성
    input_df['500버퍼'] = input_df['geometry'].iloc[0].buffer(500)
    input_df['500버퍼'] = input_df['500버퍼'].astype('geometry')
    input_df['1000버퍼'] = input_df['geometry'].iloc[0].buffer(1000)
    input_df['1000버퍼'] = input_df['1000버퍼'].astype('geometry')

    # 맛집
    food['geometry'] = food.apply(lambda row : Point([row['경도'], row['위도']]), axis= 1)
    food = gpd.GeoDataFrame(food, geometry='geometry')
    food.crs = {'init':'epsg:4326'}
    food = food.to_crs({'init':'epsg:5179'})
    f = lambda x : np.sum(food.geometry.intersects(x))
    input_df['500m맛집'] = input_df['500버퍼'].apply(f)
    f = lambda x : np.sum(food.geometry.intersects(x))
    input_df['1000m맛집'] = input_df['1000버퍼'].apply(f)

    # 문화공간
    munhwa_space['geometry'] = munhwa_space.apply(lambda row : Point([row['경도'], row['위도']]), axis= 1)
    munhwa_space = gpd.GeoDataFrame(munhwa_space, geometry='geometry')
    munhwa_space.crs = {'init':'epsg:4326'}
    munhwa_space = munhwa_space.to_crs({'init':'epsg:5179'})
    f = lambda x : np.sum(munhwa_space.geometry.intersects(x))
    input_df['500m문화공간'] = input_df['500버퍼'].apply(f)
    f = lambda x : np.sum(munhwa_space.geometry.intersects(x))
    input_df['1000m문화공간'] = input_df['1000버퍼'].apply(f)

    # 문화재
    munhwa['geometry'] = munhwa.apply(lambda row : Point([row['경도'], row['위도']]), axis= 1)
    munhwa = gpd.GeoDataFrame(munhwa, geometry='geometry')
    munhwa.crs = {'init':'epsg:4326'}
    munhwa = munhwa.to_crs({'init':'epsg:5179'})
    f = lambda x : np.sum(munhwa.geometry.intersects(x))
    input_df['500m문화재'] = input_df['500버퍼'].apply(f)
    f = lambda x : np.sum(munhwa.geometry.intersects(x))
    input_df['1000m문화재'] = input_df['1000버퍼'].apply(f)

    # 쇼핑
    shopping['geometry'] = shopping.apply(lambda row : Point([row['경도'], row['위도']]), axis= 1)
    shopping = gpd.GeoDataFrame(shopping, geometry='geometry')
    shopping.crs = {'init':'epsg:4326'}
    shopping = shopping.to_crs({'init':'epsg:5179'})
    f = lambda x : np.sum(shopping.geometry.intersects(x))
    input_df['500m쇼핑'] = input_df['500버퍼'].apply(f)
    f = lambda x : np.sum(shopping.geometry.intersects(x))
    input_df['1000m쇼핑'] = input_df['1000버퍼'].apply(f)

    # 환율
    import pandas as pd
    url ='https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query=%ED%99%98%EC%9C%A8%EC%A1%B0%ED%9A%8C'
    tables = pd.read_html(url)[0]
    USD = tables['매매기준율'].iloc[0]
    input_df['환율'] = USD

    # 최종 데이터프레임
    input_final=input_df[['단지명','전용면적','층','도로명주소','가장 가까운 지하철역까지 거리(m)','가장 가까운 지하철역',
                          '가장 가까운 버스정류장까지 거리(m)','500m맛집','1000m맛집','500m문화공간','1000m문화공간',
                          '500m문화재','1000m문화재','500m쇼핑','1000m쇼핑','환율']]

    return input_final

In [ ]:
input_df = make_input_df(input_df)
input_df = count_infra(input_df)
input_df

In [48]:

def input_your_info():
    
    # 1단계: input_df 데이터프레임 만들기
    
    def input_def():
        import pandas as pd
        global input_df
        global cnt 
        # 빈 DataFrame 생성
        # columns = ['단지명','전용면적','층','도로명주소']
        idx = [cnt]
        # heng = [['0',0.0,'0','0']]
        # input_df = pd.DataFrame(data=heng, index = idx, columns=columns )

        # 단지명 , 전용면적, 도로명주소 입력
        apart = input(' 단지명을 입력하세요, ex : 남산센트럴자이 -- ')
        myeonjuk = input(' 전용면적을 입력하세요, ex : 175 -- ')
        floor = input(' 층수를 입력하세요, ex : 21 -- ')
        juso = input(' 도로명 주소를 입력하세요, ex : 중구 퇴계로 235 -- ')

        # 단지명 : 명동교자
        # 전용면적 : 100
        # 층 : 1
        # 도로명주소 : 중구 명동10길 29
        input_df.loc[cnt] = [apart, myeonjuk, floor, juso]
        cnt +=1
        # input_df[['단지명','전용면적','층','도로명주소']][cnt] = [apart , myeonjuk, floor, juso]
        return input_df
    
    input_df=input_def()
    
    # 2단계: 프로토타입
    
    def output(input_df):
        import pandas as pd
        import numpy as np
        import geopandas as gpd
        from shapely.geometry import Point, Polygon, LineString
        import matplotlib.pyplot as plt
        import matplotlib
        import warnings
        warnings.filterwarnings('ignore')

        # 데이터 가져오기
        metro = pd.read_csv('https://raw.githubusercontent.com/8900j/hhh/main/%EC%84%9C%EC%9A%B8_%EC%A7%80%ED%95%98%EC%B2%A0_%EC%9C%84%EA%B2%BD%EB%8F%84.csv')
        bus = pd.read_csv('https://raw.githubusercontent.com/8900j/hhh/main/%EB%B2%84%EC%8A%A4%EC%A0%95%EB%A5%98%EC%9E%A5_%EC%A4%91%EA%B5%AC.csv')
        food = pd.read_csv('https://raw.githubusercontent.com/8900j/hhh/main/%EC%A4%91%EA%B5%AC_%EB%AC%B8%ED%99%94%EA%B4%80%EA%B4%91_%EB%A7%9B%EC%A7%91_%EC%9C%84%EA%B2%BD%EB%8F%84.csv')
        munhwa_space = pd.read_csv('https://raw.githubusercontent.com/8900j/hhh/main/%EC%A4%91%EA%B5%AC_%EB%AC%B8%ED%99%94%EA%B4%80%EA%B4%91_%EB%AC%B8%ED%99%94%EA%B3%B5%EA%B0%84_%EC%9C%84%EA%B2%BD%EB%8F%84.csv')
        munhwa = pd.read_csv('https://raw.githubusercontent.com/8900j/hhh/main/%EC%A4%91%EA%B5%AC_%EB%AC%B8%ED%99%94%EA%B4%80%EA%B4%91_%EB%AC%B8%ED%99%94%EC%9E%AC_%EC%9C%84%EA%B2%BD%EB%8F%84.csv')
        shopping = pd.read_csv('https://raw.githubusercontent.com/8900j/hhh/main/%EC%A4%91%EA%B5%AC_%EB%AC%B8%ED%99%94%EA%B4%80%EA%B4%91_%EC%87%BC%ED%95%91_%EC%9C%84%EA%B2%BD%EB%8F%84.csv')

        # 좌표 변환
        import requests, json
        def get_location(address):
            url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + address
            headers = {"Authorization": "KakaoAK 2550728cf88fe76f21185b7b79eddac8"}    
            try:      
                api_json = json.loads(str(requests.get(url,headers=headers).text))
                address = api_json['documents'][0]['address']
                crd = {"lat": str(address['y']), "lng": str(address['x'])}
                address_name = address['address_name']
                return crd
            except:
                crd = {"lat": 0 , "lng" : 0 }
                return crd

        input_df['위경도'] = input_df['도로명주소'].apply(lambda x : get_location(x))
        input_df['위도'] = input_df['위경도'].apply(lambda x : x['lat'])
        input_df['경도'] = input_df['위경도'].apply(lambda x : x['lng'])
        input_df.drop(columns='위경도', inplace=True)

        input_df[['위도','경도']] = input_df[['위도','경도']].astype(float)

        input_df['geometry'] = input_df.apply(lambda row : Point([row['경도'], row['위도']]), axis= 1)
        input_df = gpd.GeoDataFrame(input_df, geometry='geometry')

        input_df.crs = {'init':'epsg:4326'}
        input_df = input_df.to_crs({'init':'epsg:5179'})

        # 지하철
        metro_gdf=gpd.GeoDataFrame(metro,geometry=gpd.points_from_xy(metro.역사위치경도, metro.역사위치위도),crs='epsg:4326')
        def close_metro_dist(point_geometry):
            dist=metro_gdf.to_crs(epsg=5186).geometry.distance(point_geometry)
            return dist.min()
        def close_metro(point_geometry):
            dist=metro_gdf.to_crs(epsg=5186).geometry.distance(point_geometry)
            return metro.iloc[dist.idxmin()]['역명']
        input_df['가장 가까운 지하철역까지 거리(m)']=input_df.to_crs(epsg=5186).apply(lambda x: close_metro_dist(x.geometry),axis=1)
        input_df['가장 가까운 지하철역']=input_df.to_crs(epsg=5186).apply(lambda x: close_metro(x.geometry),axis=1)

        # 버스
        bus_gdf=gpd.GeoDataFrame(bus,geometry=gpd.points_from_xy(bus.정류장_경도, bus.정류장_위도),crs='epsg:4326')
        def close_bus_dist(point_geometry):
            dist=bus_gdf.to_crs(epsg=5186).geometry.distance(point_geometry)
            return dist.min()
        def close_bus(point_geometry):
            dist=bus_gdf.to_crs(epsg=5186).geometry.distance(point_geometry)
            return bus.iloc[dist.idxmin()]['정류장명']
        input_df['가장 가까운 버스정류장까지 거리(m)']=input_df.to_crs(epsg=5186).apply(lambda x: close_bus_dist(x.geometry),axis=1)
        input_df['가장 가까운 버스정류장']=input_df.to_crs(epsg=5186).apply(lambda x: close_bus(x.geometry),axis=1)

        # 버퍼 생성
        input_df['500버퍼'] = input_df['geometry'].iloc[0].buffer(500)
        input_df['500버퍼'] = input_df['500버퍼'].astype('geometry')
        input_df['1000버퍼'] = input_df['geometry'].iloc[0].buffer(1000)
        input_df['1000버퍼'] = input_df['1000버퍼'].astype('geometry')

        # 맛집
        food['geometry'] = food.apply(lambda row : Point([row['경도'], row['위도']]), axis= 1)
        food = gpd.GeoDataFrame(food, geometry='geometry')
        food.crs = {'init':'epsg:4326'}
        food = food.to_crs({'init':'epsg:5179'})
        f = lambda x : np.sum(food.geometry.intersects(x))
        input_df['500m맛집'] = input_df['500버퍼'].apply(f)
        f = lambda x : np.sum(food.geometry.intersects(x))
        input_df['1000m맛집'] = input_df['1000버퍼'].apply(f)

        # 문화공간
        munhwa_space['geometry'] = munhwa_space.apply(lambda row : Point([row['경도'], row['위도']]), axis= 1)
        munhwa_space = gpd.GeoDataFrame(munhwa_space, geometry='geometry')
        munhwa_space.crs = {'init':'epsg:4326'}
        munhwa_space = munhwa_space.to_crs({'init':'epsg:5179'})
        f = lambda x : np.sum(munhwa_space.geometry.intersects(x))
        input_df['500m문화공간'] = input_df['500버퍼'].apply(f)
        f = lambda x : np.sum(munhwa_space.geometry.intersects(x))
        input_df['1000m문화공간'] = input_df['1000버퍼'].apply(f)

        # 문화재
        munhwa['geometry'] = munhwa.apply(lambda row : Point([row['경도'], row['위도']]), axis= 1)
        munhwa = gpd.GeoDataFrame(munhwa, geometry='geometry')
        munhwa.crs = {'init':'epsg:4326'}
        munhwa = munhwa.to_crs({'init':'epsg:5179'})
        f = lambda x : np.sum(munhwa.geometry.intersects(x))
        input_df['500m문화재'] = input_df['500버퍼'].apply(f)
        f = lambda x : np.sum(munhwa.geometry.intersects(x))
        input_df['1000m문화재'] = input_df['1000버퍼'].apply(f)

        # 쇼핑
        shopping['geometry'] = shopping.apply(lambda row : Point([row['경도'], row['위도']]), axis= 1)
        shopping = gpd.GeoDataFrame(shopping, geometry='geometry')
        shopping.crs = {'init':'epsg:4326'}
        shopping = shopping.to_crs({'init':'epsg:5179'})
        f = lambda x : np.sum(shopping.geometry.intersects(x))
        input_df['500m쇼핑'] = input_df['500버퍼'].apply(f)
        f = lambda x : np.sum(shopping.geometry.intersects(x))
        input_df['1000m쇼핑'] = input_df['1000버퍼'].apply(f)
        
        # 환율
        import pandas as pd
        url ='https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query=%ED%99%98%EC%9C%A8%EC%A1%B0%ED%9A%8C'
        tables = pd.read_html(url)[0]
        USD = tables['매매기준율'].iloc[0]
        input_df['환율'] = USD
        
        # 최종 데이터프레임
        input_final=input_df[['단지명','전용면적','층','도로명주소','가장 가까운 지하철역까지 거리(m)','가장 가까운 지하철역',
                              '가장 가까운 버스정류장까지 거리(m)','500m맛집','1000m맛집','500m문화공간','1000m문화공간',
                              '500m문화재','1000m문화재','500m쇼핑','1000m쇼핑','환율']]

        return input_final

    return output(input_df)

# 함수 실행

In [49]:
your_final_info=input_your_info()
your_final_info

 단지명을 입력하세요, ex : 남산센트럴자이 --  1
 전용면적을 입력하세요, ex : 175 --  1
 층수를 입력하세요, ex : 21 --  1
 도로명 주소를 입력하세요, ex : 중구 퇴계로 235 --  중구 퇴계로 235


,단지명,전용면적,층,도로명주소,가장 가까운 지하철역까지 거리(m),가장 가까운 지하철역,가장 가까운 버스정류장까지 거리(m),500m맛집,1000m맛집,500m문화공간,1000m문화공간,500m문화재,1000m문화재,500m쇼핑,1000m쇼핑,환율
0,1,1,1,중구 퇴계로 235,352.701329,충무로역,129.080275,16,63,6,21,13,40,2,9,1309.4


In [50]:
your_final_info=input_your_info()
your_final_info

 단지명을 입력하세요, ex : 남산센트럴자이 --  2
 전용면적을 입력하세요, ex : 175 --  2
 층수를 입력하세요, ex : 21 --  2
 도로명 주소를 입력하세요, ex : 중구 퇴계로 235 --  중구 퇴계로 235


ValueError: cannot set a row with mismatched columns

In [32]:
# 보여지는 칼럼수 1000개 로 늘이기
pd.set_option('display.max_columns', 1000)


In [33]:
data = pd.read_csv('dt_train.csv')
data.head()

,단지명,전용면적,계약년도,계약일,위도,경도,지하철거리,계약날짜,환율,500m맛집,500m문화공간,500m문화재,500m쇼핑,1000m맛집,1000m문화공간,1000m문화재,1000m쇼핑,가장 가까운 버스정류장까지 거리(m),가장 가까운 버스정류장,일일가격,층_-1,층_1,층_2,층_3,층_4,층_5,층_6,층_7,층_8,층_9,층_10,층_11,층_12,층_13,층_14,층_15,층_16,층_17,층_18,층_19,층_20,층_21,층_22,층_23,층_24,층_25,층_26,층_27,층_28,층_29,층_30,층_31,층_32,층_33,층_34,지하철역_동대입구역,지하철역_동묘앞역,지하철역_명동역,지하철역_버티고개역,지하철역_상왕십리역,지하철역_서대문역,지하철역_서울역,지하철역_시청역,지하철역_신금호역,지하철역_신당역,지하철역_신설동역,지하철역_약수역,지하철역_을지로4가역,지하철역_청구역,지하철역_충무로역,지하철역_충정로(경기대입구)역,지하철역_회현(남대문시장)역
0,현대,79.11,2022-03-01,28,37.560504,127.022971,642.530414,2022/03/28,1218.3,3,0,1,0,27,9,13,16,145.015674,금호베스트빌.래미안하이베르,8.680556,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,래미안하이베르,59.95,2022-09-01,2,37.561038,127.020986,524.953495,2022/09/02,1351.4,7,0,1,0,44,15,23,27,264.007733,동화동,33.541667,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,남산롯데캐슬아이리스,103.49,2022-05-01,27,37.558225,126.981744,310.774962,2022/05/27,1266.4,27,7,12,5,163,48,118,11,85.659904,남산3호터널,75.243056,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,쌍용남산플래티넘,166.24,2022-05-01,6,37.559412,126.983220,325.431248,2022/05/06,1261.8,43,5,10,3,175,51,123,11,53.132052,남산3호터널.서울애니메이션센터,53.993056,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,충무,67.77,2022-01-01,27,37.560732,127.000855,458.952760,2022/01/27,1197.0,16,9,15,5,107,37,58,32,159.189499,앰버서더호텔,15.069444,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [38]:
df1 = pd.DataFrame(columns=range(4))
df1.columns = ['단지명','전용면적','층','도로명주소']
df1.append([1,2,3,4])

,단지명,전용면적,층,도로명주소,0
0,NaN,NaN,NaN,NaN,1.0
1,NaN,NaN,NaN,NaN,2.0
2,NaN,NaN,NaN,NaN,3.0
3,NaN,NaN,NaN,NaN,4.0
